In [ ]:
from LoopStructural import GeologicalModel
from LoopStructural.visualisation import LavaVuModelViewer
# import other libraries
import pandas as pd
import numpy as np
from scipy.interpolate import Rbf
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import lavavu
from pyamg import solve
import os
from map2loop import m2l_export

%load_ext autoreload

### Load and process the data
We need to create a dataset that has the following columns: **type, X, Y, Z**, val, strike, dip, polarity, nx, ny, nz.
The bold columns are required and the unbold ones are optional.

Val should be the relative scalar field value of the contacts from a reference horizon. If the base of a series is 0, val should be the accumulated thickness. 

In this example we will build the scalar field for one series. The order and names of the different units in the series components were extracted map2loop topology analysis and are for now hardcoded.  

In [ ]:
test_data_name='test_data3'

test_data_path='../'+test_data_name+'/'

os.chdir(test_data_path)
%run -i "m2l_config.py"
print(os.getcwd())

bbox=(minx,miny,maxx,maxy)


In [ ]:
m2l_export.loop2LoopStructural(output_path+'formation_thicknesses.csv',output_path+'orientations.csv',output_path+'contacts4.csv',bbox)

In [ ]:
import winsound
duration = 700  # milliseconds
freq = 1100  # Hz
winsound.Beep(freq, duration)
winsound.Beep(freq, duration)
winsound.Beep(freq, duration)
